In [1]:
import undetected_chromedriver
import time
import re
import random
# from datetime import datetime, date, time, timedelta
from translate import Translator
import pickle
import requests
from requests.auth import HTTPBasicAuth
import time

from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys 

import numpy as np
import pandas as pd
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
class Russian_romanizer(object):

    """преобразует русское название города в латиницу,
    которая должна подходить для названий городов в яндексе"""
    cyrillic = {
        u'\u0401': u'YO',
        u'\u0410': u'a',
        u'\u0411': u'B',
        u'\u0412': u'V',
        u'\u0413': u'G',
        u'\u0414': u'D',
        u'\u0415': u'E',
        u'\u0416': u'ZH',
        u'\u0417': u'Z',
        u'\u0418': u'I',
        u'\u0419': u'I',
        u'\u041a': u'K',
        u'\u041b': u'L',
        u'\u041c': u'M',
        u'\u041d': u'N',
        u'\u041e': u'O',
        u'\u041f': u'P',
        u'\u0420': u'R',
        u'\u0421': u'S',
        u'\u0422': u'T',
        u'\u0423': u'U',
        u'\u0424': u'F',
        u'\u0425': u'H',
        u'\u0426': u'TS',
        u'\u0427': u'CH',
        u'\u0428': u'SH',
        u'\u0429': u'SHCH',
        u'\u042a': u"",
        u'\u042b': u'I',
        u'\u042c': u"", 
        u'\u042d': u'E',
        u'\u042e': u'YU',
        u'\u042f': u'YA',
        u'\u0430': u'a',
        u'\u0431': u'b',
        u'\u0432': u'v',
        u'\u0433': u'g',
        u'\u0434': u'd',
        u'\u0435': u'e', 
        u'\u0436': u'zh',
        u'\u0437': u'z',
        u'\u0438': u'i',
        u'\u0439': u'i', 
        u'\u043a': u'k',
        u'\u043b': u'l',
        u'\u043c': u'm',
        u'\u043d': u'n',
        u'\u043e': u'o',
        u'\u043f': u'p',
        u'\u0440': u'r',
        u'\u0441': u's',
        u'\u0442': u't',
        u'\u0443': u'u',
        u'\u0444': u'f',
        u'\u0445': u'kh', 
        u'\u0446': u'ts',
        u'\u0447': u'ch',
        u'\u0448': u'sh',
        u'\u0449': u'shch',
        u'\u044a': u"", 
        u'\u044b': u'y',
        u'\u044c': u"", 
        u'\u044d': u'e', 
        u'\u044e': u'yu',
        u'\u044f': u'ya',
        u'\u0451': u'yo',
    }
    
    def __init__(self,txt):
        self.txt = txt
        
    def from_cyrillic(self,char):
        if char in self.cyrillic:
            return self.cyrillic[char]
        else:
            return char
        
    def transliterate(self):
        return ''.join([self.from_cyrillic(val) for val in self.txt])

In [57]:
with open(f'../cities.txt') as f:
    cities = [x.strip('\n') for x in f ]
len(cities)

633

In [58]:
def romanize_list(name):
    rom = Russian_romanizer(name)
    return rom.transliterate()


def translate_from_eng(word):
    translator = Translator(from_lang='english', to_lang='russian')
    r = translator.translate(word)
    # time.sleep(10)
    return r

cities.extend([romanize_list(c) for c in cities])
len(cities)

1266

In [59]:
df = pd.read_csv('yandex_regions.csv', sep=';', encoding='Windows-1251', header=None, names=['city_code', 'city_name', 'country', 'coordinates'])
df.sample()

,city_code,city_name,country,coordinates
21466,131296,Суздальская,"Городской округ Горячий Ключ, Краснодарский кр...","39.370509,44.762645"


In [61]:
%%time
df['roman'] = None
df['russian_country'] = None
indexes = []

for row in df.itertuples():
    
    if isinstance(row.country , str) and ('Россия' in row.country or 'Russia' in row.country):
        if row.city_name in cities:
            indexes.append(row.Index)
    
        rom = Russian_romanizer(row.city_name)
        df.loc[row.Index, 'roman'] = rom.transliterate()

df2 = df.loc[indexes]
df2['roman'] = df2['roman'].str.lower()
df2.sample(5)

CPU times: user 1.27 s, sys: 12.3 ms, total: 1.28 s
Wall time: 1.29 s


,city_code,city_name,country,coordinates,roman,russian_country
750,11022,Моздок,"Республика Северная Осетия — Алания, Россия","44.657058,43.747195",mozdok,None
1500,20689,Бирск,"Республика Башкортостан, Россия","55.540471,55.412523",birsk,None
15128,117570,Волжский,"Енотаевский район, Астраханская область, Россия","47.548717,46.968312",volzhskii,None
146,1107,Анапа,"Краснодарский край, Россия","37.316259,44.894997",anapa,None
753,11026,Гудермес,"Чеченская Республика, Россия","46.103535,43.35188",gudermes,None


In [62]:
keys = []
for row in df2.itertuples():
    if isinstance(row.country , str) and 'Russia' in row.country:
        keys.append(row.country)

In [63]:
values = ["Ивановская область, Россия",
"Липецкая область, Россия",
"Рязанская область, Россия",
"Смоленская область, Россия",
"Тамбовская область, Россия",
"Тульская область, Россия",
"Республика Карелия, Россия",
"Вологодская область, Россия",
"Мурманская Область, Россия",
"Псковская область, Россия",
"Ставропольский край, Россия",
"Ростовская область, Россия",
"Удмуртская Республика, Россия",
"Пензенская область, Россия",
"Пермский край, Россия",
"Иркутская область, Россия",
"Омская область, Россия",
"Томский район, Россия",
"Воронежская область, Россия",
"Москва и Московская область, Россия",
"Краснодарский край, Россия",
"Калужская область, Россия",
 "Санкт-Петербург и Ленинградская область, Россия",
"Ростовская область, Россия",
"Нижегородская область, Россия",
"Краснодарский край, Россия",
"Республика Калмыкия, Россия",
"Брянская область, Россия",
"Владимирская область, Россия",
"Владимирская область, Россия",
"Владимирская область, Россия",
"Воронежская область, Россия",
"Москва и Московская область, Россия",
"Москва и Московская область, Россия",
"Москва и Московская область, Россия",
"Москва и Московская область, Россия",
"Москва и Московская область, Россия",
"Москва и Московская область, Россия",
"Москва и Московская область, Россия",
"Рязанская область, Россия",
 "Рязанская область, Россия",
"Смоленская область, Россия",
"Смоленская область, Россия",
"Смоленская область, Россия",
"Смоленская область, Россия",
"Тверская область, Россия",
"Тверская область, Россия",
"Ярославская область, Россия",
 "Архангельская область, Россия",
"Санкт-Петербург и Ленинградская область, Россия",
"Санкт-Петербург и Ленинградская область, Россия",
"Санкт-Петербург и Ленинградская область, Россия",
"Мурманская область, Россия",
"Мурманская область, Россия",
"Новгородская область, Россия",
"Республика Карелия, Россия",
"Республика Карелия, Россия",
"Республика Коми, Россия",
 "Республика Коми, Россия",
"Ростовская область, Россия",
"Ростовская область, Россия",
"Ростовская область, Россия",
"Ростовская область, Россия",
"Ставропольский край, Россия",
"Ставропольский край, Россия",
"Нижегородская область, Россия",
"Оренбург Область, Россия",
"Республика Башкортостан, Россия",
"Республика Башкортостан, Россия",
"Саратовская область, Россия",
"Удмуртская Республика, Россия",
"Удмуртская Республика, Россия",
"Ульяновская область, Россия",
"Свердловская область, Россия",
"Свердловская область, Россия",
"Тюменская область, Россия",
"Ханты-Мансийский автономный округ — Югра, Россия",
 "Челябинская область, Россия",
"Челябинская область, Россия",
"Челябинская область, Россия",
"Челябинская область, Россия",
"Алтайский край, Россия",
"Алтайский край, Россия",
"Иркутская область, Россия",
"Иркутская область, Россия",
"Кемеровская область, Россия",
"Кемеровская область, Россия",
"Кемеровская область, Россия",
"Кемеровская область, Россия",
"Красноярский край, Россия",
"Республика Хакасия, Россия",
"Томская область, Россия",
"Томский район, Россия",
"Приморский край, Россия",
"Приморский край, Россия",
"Сахалинская область, Россия",
"Республика Мордовия, Россия",
 "Нижегородская область, Россия",
"Чувашская Республика, Россия",
"Новосибирская область, Россия",
"Новосибирская область, Россия",
"Омская область, Россия",
"Омская область, Россия",
"Мурманская область, Россия",
"Калужская область, Россия",
"Липецк Область, Россия",
"Пермский край, Россия",
"Москва и Московская область, Россия",
"Курская область, Россия",
"Республика Башкортостан, Россия",
"Свердловская область, Россия",
"Москва и Московская область, Россия",
"Ханты-Мансийский автономный округ — Югра, Россия",
"Ивановская область, Россия",
"Краснодарский край, Россия",
"Дзержинский район, Красноярский край, Россия",
 "Тамбовская область, Россия",
"Республика Марий Эл, Россия",
"Республика Дагестан, Россия",
"Москва и Московская область, Россия",
"Москва и Московская область, Россия",
"Москва и Московская область, Россия",
"Москва и Московская область, Россия",
"Сосновоборский район, Пензенская область, Россия",
"Березовский район, Ханты-Мансийский автономный округ — Югра, Россия",
"Санкт-Петербург и Ленинградская область, Россия",
"Иркутский район, Иркутская область, Россия",
"Кировский район, Санкт-Петербург и Ленинградская область, Россия",
"Городской округ Североморск, Мурманская область, Россия",
 "Городской округ Домодедово, Москва и Московская область, Россия",
"Мезенский район, Архангельская область, Россия",
"Шаранский район, Республика Башкортостан, Россия",
"Покровский район, Орловская область, Россия",
"Хотынецкий район, Орловская область, Россия",
"Вадинский район, Пензенская область, Россия",
"Каширский район, Ростовская область, Россия",
"Миллеровский район, Ростовская область, Россия",
"Монастырщинский район, Смоленская область, Россия",
"Лев-Толстовский район, Липецкая область, Россия",
"Вачский район, Нижегородская область, Россия",
 "Лискинский район, Воронежская область, Россия",
"Добринский район, Липецкая область, Россия",
"Верховский район, Орловская область, Россия",
"Раменский район, Москва и Московская область, Россия",
"Угранский район, Смоленская область, Россия",
"Хвойнинский район, Новгородская область, Россия",
"Аургазинский район, Республика Башкортостан, Россия"]

In [64]:
N = 146
keys[N], values[N]

('Aurgazinsky District, Republic of Bashkortostan, Russia',
 'Аургазинский район, Республика Башкортостан, Россия')

In [65]:
city_dict = dict(zip(keys, values))

In [66]:
# df2['country'] = np.where('Russia' in df2['country'], df2['country'].map(city_dict), df2['country'])

for row in df2.itertuples():
    if isinstance(row.country , str) and 'Russia' in row.country:
        df2.loc[row.Index, 'country'] = city_dict[row.country]

In [73]:
df2['country'] = df2['country'].str.replace(', Russia', '').str.replace(', Россия', '').strip()
df2.sample(4)

,city_code,city_name,country,coordinates,roman,russian_country
15155,117632,Вязники,"Шпаковский район, Ставропольский край","41.975282,45.110294",vyazniki,None
1077,11436,Мирный,Республика Саха (Якутия),"113.978692,62.541028",mirnyi,None
137,976,Братск,Иркутская область,"101.63408,56.151362",bratsk,None
20418,128964,Горячий Ключ,"Иркутский район, Иркутская область","104.909122,52.276084",goryachii klyuch,None


In [76]:
ids = df2["city_name"]
duplicated = df2[ids.isin(ids[ids.duplicated()])].sort_values("city_name")
len(np.unique(duplicated['city_name']))

101

In [ ]:
city_region = pd.read_excel('../city_region.xlsx')
city_region['region'] = city_region['region'].str.strip()
city_region.sample(3)

In [81]:
indexes_to_drop = []

for row in duplicated.itertuples():
    
    if isinstance(row.country , str) and row.country not in needed_regions:
        indexes_to_drop.append(row.Index)
len(indexes_to_drop)

303

In [86]:
needed_indexes = set(df2.index) - set(indexes_to_drop)
df2 = df2.loc[needed_indexes]

In [91]:
# TODO: 
df2['code'] = df2['city_code'].astype(str) + '/' + df2['roman'].astype(str)
df2.sample()

,city_code,city_name,country,coordinates,roman,russian_country,code
134,973,Сургут,Ханты-Мансийский автономный округ - Югра,"73.396204,61.254052",surgut,None,973/surgut


In [92]:
d = {}
df_list = df2[['city_name', 'code']].to_dict('record')

for i in df_list:
    d[i['city_name']] = i['code']

In [94]:
with open(f'../cities_code_dict.pickle', 'rb') as handle:
    prev_d = pickle.load(handle)

prev_d.update(d)

with open(f'../cities_code_dict.pickle', 'wb') as handle:
    pickle.dump(d, handle)


In [47]:
with open(f'cities_code_dict.pickle', 'rb') as handle:
    prev_d = pickle.load(handle)
prev_d

{'Белгород': '155820/belgorod',
 'Калуга': '138691/kaluga',
 'Кострома': '162604/kostroma',
 'Курск': '137946/kursk',
 'Архангельск': '20/arkhangelsk',
 'Калининград': '22/kaliningrad',
 'Великий Новгород': '24/velikii novgorod',
 'Махачкала': '28/makhachkala',
 'Нальчик': '30/nalchik',
 'Краснодар': '154382/krasnodar',
 'Астрахань': '37/astrakhan',
 'Волгоград': '38/volgograd',
 'Йошкар-Ола': '41/ioshkar-ola',
 'Саранск': '42/saransk',
 'Казань': '43/kazan',
 'Чебоксары': '169234/cheboksary',
 'Киров': '157654/kirov',
 'Нижний Новгород': '47/nizhnii novgorod',
 'Оренбург': '48/orenburg',
 'Самара': '153277/samara',
 'Курган': '166258/kurgan',
 'Екатеринбург': '54/ekaterinburg',
 'Челябинск': '56/chelyabinsk',
 'Салехард': '58/salekhard',
 'Красноярск': '62/krasnoyarsk',
 'Кемерово': '64/kemerovo',
 'Новосибирск': '65/novosibirsk',
 'Чита': '141081/chita',
 'Хабаровск': '76/habarovsk',
 'Благовещенск': '11112/blagoveshchensk',
 'Петропавловск-Камчатский': '78/petropavlovsk-kamchatskii'

In [51]:
# indexes = []
# for i in cities:
#     last_w = i.split(' ')[-1]
#     if len(df[df['city_name'] == last_w]) > 0:
#         indexes.append(df.Index)

In [66]:
df[df['city_name'] == 'Sochi']

,city_code,city_name,country,coordinates,roman
118,239,Sochi,"Krasnodar Krai, Russia","39.723062,43.585525",Sochi


In [67]:
df[df['city_name'] == 'Smolensk']

,city_code,city_name,country,coordinates,roman
9,12,Smolensk,"Smolensk Oblast, Russia","32.045134,54.78264",Smolensk


In [57]:
df2[df2['city_name'] == 'Сочи']

,city_code,city_name,country,coordinates,roman,code


In [89]:
latin_name = df[df['country'].str.contains('Russia', na=False)]
# latin_name[latin_name['city_name'] == 'Severodvinsk']
latin_name.sample(4)

,city_code,city_name,country,coordinates
19585,127061,Edon,"Vyaznikovsky District, Vladimir Oblast, Russia","41.752589,56.094093"
22182,132930,Olkhovka,"Dobrinsky District, Lipetsk Oblast, Russia","40.096976,52.202444"
34221,163569,Lipovets,"Livensky District, Oryol Oblast, Russia","37.695366,52.44133"
18585,124996,Pochtovoye,"Kholmogorsky District, Arkhange'lskaya Oblast'...","42.017062,63.157801"


In [91]:
russian_name[russian_name['city_name'] == 'ст. Талица']

,city_code,city_name,country,coordinates


In [97]:
df[df['city_name'].str.contains('Promyshlennaya', na=False)]

,city_code,city_name,country,coordinates
973,11293,Promyshlennaya,"Promyshlennovsky District, Kemerovo Oblast, Ru...","85.637654,54.916511"


In [90]:
# russian_name = df[df['country'].str.contains('Россия', na=False)]
russian_name

,city_code,city_name,country,coordinates
1,4,Белгород,Россия,"36.587223,50.59566"
3,6,Калуга,Россия,"36.252277,54.507014"
4,7,Кострома,Россия,"40.926858,57.767961"
5,8,Курск,Россия,"36.192647,51.730361"
16,20,Архангельск,Россия,"40.516939,64.539393"
...,...,...,...,...
36135,169478,Тупик 9 км,"Зубово-Полянский район, Республика Мордовия, Р...","42.330601,54.201587"
36136,169479,Первое Мая,"Зубово-Полянский район, Республика Мордовия, Р...","43.006413,53.946403"
36137,169482,Исаевка,"Зубово-Полянский район, Республика Мордовия, Р...","42.844222,54.033823"
36138,169483,Красный Лундан,"Зубово-Полянский район, Республика Мордовия, Р...","42.84265,54.051149"


In [56]:
len(latin_name), len(russian_name), len(df)

(4129, 17704, 36141)

In [79]:
# df[~ ((df['country'].str.contains('Россия', na=False)) | (df['country'].str.contains('Russia', na=False)) )].sample(10)

In [54]:
df[~df['country'].str.contains('Россия', na=False)].sample(9)

,city_code,city_name,country,coordinates
21319,130977,Kabanovo,"Galichsky District, Kostroma Oblast, Russia","42.00048,58.264596"
12793,113712,Лоувилл,"Штат Нью-Йорк, США","-75.490464,43.786946"
16519,120195,Tokarevo,"Lyubertsy Urban Okrug, Moscow and Moscow Oblas...","37.91893,55.628037"
5208,101275,Smidovich,"Smidovichsky District, Jewish Autonomous Oblas...","133.811113,48.594691"
18555,124943,Verkhnyaya Zolotitsa,"Primorsky region, Arkhange'lskaya Oblast', Russia","40.366426,65.688367"
1556,20766,Агуадилья,Пуэрто-Рико,"-67.155185,18.4346"
22370,133282,Anisimovo,"Koverninsky District, Nizhny Novgorod Oblast',...","44.099842,57.05678"
202,10261,Dilijan,"Tavush, Armenia","44.87189,40.741983"
15751,118612,Leskolovo,"Vsevolozhsk District, Saint-Petersburg and Len...","30.463605,60.271815"


In [3]:
def get_yndx_city_code(num):

    # print(num)
    """подставляю в url номер num определяет, для какого города в яндекс картах этот номер является id-шникос"""
    

    opts = undetected_chromedriver.ChromeOptions()
    opts.add_argument("--disable-renderer-backgrounding")
    opts.add_argument("--disable-extensions")
    opts.add_argument('--no-sandbox')
    opts.add_argument('--disable-dev-shm-usage')
    opts.add_argument('headless')
    opts.add_argument('--disable-gpu')
    driver = undetected_chromedriver.Chrome(options=opts)
    
    url = f'https://yandex.ru/maps/{num}/ivanovo'
    
    driver.get(url)
    N = round(random.uniform(7.1, 9.9), 2)
    time.sleep(N)

    s = driver.current_url
    
    if s != url:
        print(n, s.split('/')[5])
        return s.split('/')[5]
        
    
    driver.close()
    driver.quit()

In [6]:
with open(f'cities_code_dict.pickle', 'rb') as handle:
    prev_d = pickle.load(handle)
prev_d

{75: 'vladivostok'}

In [7]:
d = {}
for n in list(range(100, 120)):
    """функция собираед id городов в яндекс картах и записывает в словарь в cities_code_dict."""
    d[n] = get_yndx_city_code(n)

    with open(f'cities_code_dict.pickle', 'rb') as handle:
        prev_d = pickle.load(handle)

    prev_d.update(d)

    with open(f'cities_code_dict.pickle', 'wb') as handle:
        pickle.dump(d, handle)


    N = round(random.uniform(13.1, 19.9), 2) # was higher
    time.sleep(N)

100 frankfurt-am-main
101 stuttgart
102 united-kingdom


KeyboardInterrupt: 

In [4]:
with open(f'cities_code_dict.pickle', 'rb') as handle:
    cities_code_dict = pickle.load(handle)
cities_code_dict

{75: 'vladivostok'}

In [15]:
city_name = 'voronezh'
with open(f'links_{city_name}.pickle', 'rb') as handle:
    city_links = pickle.load(handle)
len(city_links), len(set(city_links))

(490, 490)

In [104]:
# %%time
# opts = undetected_chromedriver.ChromeOptions()
# opts.add_argument("--disable-renderer-backgrounding")
# opts.add_argument("--disable-extensions")
# opts.add_argument('--no-sandbox')
# opts.add_argument('--disable-dev-shm-usage')
# opts.add_argument('headless')
# opts.add_argument('--disable-gpu')
# driver = undetected_chromedriver.Chrome(options=opts)

# # TODO: chain / related как опция


# # city_code = "213/moscow/"
# # city_code = "1095/abakan"
# # city_code = "10940/vorkuta"
# city_code = '193/voronezh'

# city_name = city_code.split('/')[1]
# url = f"https://yandex.ru/maps/{city_code}/category/bank/"

# driver.get(url)

# # N = round(random.uniform(13.1, 19.9), 2) # was higher
# # print(f'sleep for {N}')
# # time.sleep(N)

# # t = 'business-tab-wrapper'  # вот эта более корректная? 
# # t = 'search-business-snippet-view'
# t = 'search-snippet-view'
# elements = driver.find_elements(By.CLASS_NAME, t)
# print(f'LEN elements at start {len(elements)}')

# links = set()

# if len(elements) >= 1:

    
#     # без этого не работает
#     elements = driver.find_elements(By.CLASS_NAME, t)
#     print(f'LEN elements after {len(elements)}')

#     seen = []

#     while True:
#         # скролл
#         driver.execute_script("arguments[0].scrollIntoView();", elements[-1]);

#         N = round(random.uniform(13.1, 19.9), 2) # was higher
#         print(f'sleep for {N}')
#         time.sleep(N)
        
#         elements = driver.find_elements(By.CLASS_NAME, t)
        
#         if len(elements) < 1:
#             T = round(random.uniform(10.1, 19.9), 2)
#             print(f'sleep again for {T}')
#             time.sleep(T)


#         for i, e in enumerate(elements):
#             print(f'* {i} *')
#             link = e.find_element(By.XPATH, ".//a[@class='link-overlay']").get_attribute('href')
#             # link = e.find_element(By.XPATH, ".//a[@class='search-snippet-view__link-overlay _focusable']").get_attribute('href')
#             print('link', link)
#             links.add(link)

#         last_len = len(elements)
#         print('LAST LEN ', last_len)
#         seen.append(last_len)
#         print('SEEN ', seen)

#         with open(f'links_{city_name}.pickle', 'wb') as handle:
#             pickle.dump(links, handle)

#         if len(seen) > 2 and seen[-2] == last_len:
#             print('**')
#             break


        
# driver.close()
# driver.quit() 

In [5]:
# перезаписывать линки 
with open(f'cities_code_dict.pickle', 'rb') as handle:
    cities_code_dict = pickle.load(handle)
len(cities_code_dict)

9

In [6]:
cities_code_dict

{37: 'astrahan',
 38: 'volgograd',
 39: 'rostov-na-donu',
 40: 'volga-federal-district',
 41: 'yoshkar-ola',
 42: 'saransk',
 43: 'kazan',
 44: 'izhevsk',
 45: None}

In [8]:
with open(f'links_abakan.pickle', 'rb') as handle:
    links = pickle.load(handle)
len(links)

52

In [10]:
[l for l in links if 'alfa' in l]

['https://yandex.ru/maps/org/alfa_bank/95773847314/']

In [11]:
[l for l in links if 'sber' in l]

['https://yandex.ru/maps/org/sberbank_rossii/1108393619/',
 'https://yandex.ru/maps/org/sberbank/129172296119/',
 'https://yandex.ru/maps/org/sberbank/1612348941/',
 'https://yandex.ru/maps/org/sberpervy/1657685442/',
 'https://yandex.ru/maps/org/sberbank/142447951722/',
 'https://yandex.ru/maps/org/sberbank/1661772449/',
 'https://yandex.ru/maps/org/sberbank/2432536431/',
 'https://yandex.ru/maps/org/sberbank/201279908503/',
 'https://yandex.ru/maps/org/sberbank/139342182959/']

In [12]:
[l for l in links if 'sberbank' in l or 'sberpervy' in l]

['https://yandex.ru/maps/org/sberbank_rossii/1108393619/',
 'https://yandex.ru/maps/org/sberbank/129172296119/',
 'https://yandex.ru/maps/org/sberbank/1612348941/',
 'https://yandex.ru/maps/org/sberpervy/1657685442/',
 'https://yandex.ru/maps/org/sberbank/142447951722/',
 'https://yandex.ru/maps/org/sberbank/1661772449/',
 'https://yandex.ru/maps/org/sberbank/2432536431/',
 'https://yandex.ru/maps/org/sberbank/201279908503/',
 'https://yandex.ru/maps/org/sberbank/139342182959/']